# Non linear optimization: Newton local

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

from collections.abc import Callable

import numpy as np
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


In this lab, you will practice **Newton’s local method** on a simple function to see
how the quadratic model, gradient, and Hessian work together. You will (i) implement a function
that returns f, ∇f, and ∇²f; (ii) build the **quadratic model** around a point and plot it; (iii)
compute **Newton’s direction** by solving ∇²f(x) d = −∇f(x) and the resulting **Newton point**;
and (iv) compare it with **Cauchy’s point** from steepest descent. The goal is to connect the
algebra (derivatives and linear solves) with the geometry (level sets and surfaces) and to see
why, for a convex quadratic, Newton’s local method reaches the minimizer in a **single iteration**.

Consider the function $f:\mathbb{R}^2 \to \mathbb{R}$ defined as
$$
f(x)=\frac{9}{2} x_1^2 + \frac{1}{2}x_2^2.
$$

The derivatives are:
$$
\nabla f(x)=\left(\begin{array}{c}9 x_1 \\ x_2 \end{array}\right),\;
\nabla^2 f(x) =
\begin{pmatrix*}
9 & 0 \\ 0 & 1
\end{pmatrix*}.
$$

#  Question 1
Implement a Python function that calculates the function above, as well as its gradient and second derivatives
matrix. Test it with $x=(1,1)^T$.

In [ ]:
def objective_function(x: np.array) -> tuple[float, np.array, np.array]:
    """Calculates the objective function and its derivatives

    :param x: an array of dimension 2
    :return: a tuple with the value of the function, the gradient and the second derivatives matrix
    """
    if len(x) != 2:
        error_msg = f'A vector of size 2 is requested. Not {len(x)}'
        raise ValueError(error_msg)

    f = (9.0 / 2.0) * x[0] * x[0]
    g = np.array([9 * x[0], x[1]])
    h = np.array([[9, 0], [0, 1]])
    return f, g, h



Test the function

In [ ]:
x = np.array([1, 1])
function, gradient, hessian = objective_function(x)
print(f'f(x)={function}')
print(f'gradient(x)={gradient}')
print(f'hessian(x)={hessian}')



We implement a function that plots a function with two variables.

In [ ]:
def plot(a_function: Callable[[np.array], float]) -> None:
    """
    Plot a function in 3D.

    :param a_function: the function to plot
    """
    x1_values = np.linspace(-10, 10, 100)
    x2_values = np.linspace(-10, 10, 100)
    X1, X2 = np.meshgrid(x1_values, x2_values)
    Y = np.array(
        [
            [a_function(np.array([x1, x2])) for x1, x2 in zip(row_x1, row_x2)]
            for row_x1, row_x2 in zip(X1, X2)
        ]
    )
    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X1, X2, Y, cmap='viridis')

    ax.set_xlabel('x_1')
    ax.set_ylabel('x_2')
    plt.show()



In order to plot, we need a function that returns only the value of the function, not the derivatives.

In [ ]:
def function_to_plot(x: np.array) -> float:
    """Function to be plotted. We ignore the derivatives

    :param x: an array of dimension 2
    :return: f(x)
    """
    value, _, _ = objective_function(x)
    return value



In [ ]:
plot(function_to_plot)



# Question 2
Implement a Python function that calculates the quadratic model of the function at a point `x_plus`.

The quadratic model of the function at $x^+$ is
\begin{align*}
m_{x^+} (x) &= f(x^+) + (x-x^+)^T \nabla f(x^+) + \frac{1}{2}
(x-x^+)^T \nabla^2 f(x^+) (x-x^+), \\
&= 5 + 9 (x_1-1) +  (x_2 -1) + \frac{1}{2}(x_1-1)(9x_1 -9)+
\frac{1}{2} (x_2-1)(x_2-1) \\
&= \frac{9}{2} x_1^2 + \frac{1}{2} x_2^2.
\end{align*}
Actually, the objective function being already quadratic, the
quadratic model is the function itself.

In [ ]:
def quadratic_model(x: np.array, x_plus: np.array) -> float:
    """Calculates the quadratic model around a point.

    :param x: an array of dimension 2
    :param x_plus: an array of dimension 2 with the point around which the model is built.
    :return: the value of the quadratic model.
    """
    the_function, the_gradient, the_hessian = objective_function(x_plus)
    the_model = (
        the_function
        + np.dot(x - x_plus, the_gradient)
        + 0.5 * np.dot(x - x_plus, the_hessian @ (x - x_plus))
    )
    return the_model



We plot the model around the point $(1,1)$.

In [ ]:
def model_to_plot(x: np.array) -> float:
    """Function to be plotted.

    :param x: an array of dimension 2
    :return: f(x)
    """
    x_plus = np.array([1, 1])
    value = quadratic_model(x, x_plus)
    return value


plot(model_to_plot)


# Question 3
Calculates Newton's point at $(1, 1)$.

In [ ]:
x = np.array([1, 1])
function, gradient, hessian = objective_function(x)


We solve the system of equations (Definition 10.3, Eq. (10.14) p. 243)
\begin{align*}
\nabla^2 f(x^+) d &= -\nabla f(x^+), \\
\left(\begin{array}{c}9 d_1 \\d_2\end{array}\right) &= -\left(\begin{array}{c}9 \\ 1\end{array}\right).
\end{align*}

In [ ]:
direction = np.linalg.solve(hessian, -gradient)
print(direction)


The solution is $d_1=-1$, $d_2=-1$. Therefore, Newton's point is
$$
x_N = \left(\begin{array}{c}1\\1\end{array}\right) + \left(\begin{array}{c}-1 \\-1\end{array}\right) =
\left(\begin{array}{c}0\\0\end{array}\right).
$$

In [ ]:
newton_point = x + direction
print(newton_point)


# Question 4
Calculates Cauchy's point at $(1, 1)$.

We need to consider the quadratic model in the direction of
the gradient, that is at a point
$$
x_\alpha= \left(\begin{array}{c}1-9\alpha \\ 1-\alpha\end{array}\right).
$$
We have
\begin{align*}
m_{x^+} (x_\alpha) &= \frac{9}{2}(1-9\alpha)^2 + \frac{1}{2} (1-\alpha)^2, \\
&= 365 \alpha^2 - 82 \alpha + 5.
\end{align*}

As $f$ is convex, the minimum can be obtained directly from
Eq. (10.17) on page 244.

In [ ]:
alpha_cauchy = np.dot(gradient, gradient) / np.dot(gradient, hessian @ gradient)
print(f"Step to obtain Cauchy's point: {alpha_cauchy:.4g}")


And we calculate Cauchy's point along the steepest descent direction.
Using Definition 10.4, Eq.(10.15), p. 243:
$$
x_C = \left(\begin{array}{c}1 \\ 1\end{array}\right)
- \frac{41}{365} \left(\begin{array}{c}9\\1\end{array}\right) =
\left(\begin{array}{c}-0.01096
\\ 0.8877\end{array}\right).
$$

In [ ]:
cauchy_point = x - alpha_cauchy * gradient
print(cauchy_point)


# Question 5
Perform one iteration of Newton's local method.

An iteration of Newton's local method calculates Newton's
point. We have already calculated it:
$$
x_N = \left(\begin{array}{c}0\\0\end{array}\right).
$$

In [ ]:
new_iterate = newton_point
print(new_iterate)


In [ ]:
f, gradient, hessian = objective_function(new_iterate)
print(f'{f=}')
print(f'{gradient=}')
print(f'{hessian=}')


We note that the gradient is zero at that point. As the second
derivative matrix is positive definite, we have found a minimum of
the problem (it is actually the unique minimum). It illustrates an
interesting property of Newton's local method: if the objective function is a
convex quadratic function, Newton's local method converges in only
one iteration.